In [ ]:
# --- BLOQUE 1: INSTALACIÓN ---
print("⏳ Instalando librerías...")
!pip install -U transformers datasets accelerate

import torch
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    DataCollatorForLanguageModeling, 
    GPT2LMHeadModel, 
    TrainingArguments, 
    Trainer
)

# Verificamos GPU
if torch.cuda.is_available():
    print(f"✅ GPU Detectada: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ ¡OJO! No detecto GPU. Revisa los 'Session Options' a la derecha.")

print("✅ BLOQUE 1 TERMINADO: Librerías instaladas.")

In [ ]:
# --- BLOQUE 2: CARGAR DATOS ---
# Ruta exacta basada en tu captura
dataset_file = "/kaggle/input/dataset-clean-v2/dataset_TRAVIAN_CLEAN_V2.txt"

if os.path.exists(dataset_file):
    print(f"✅ Archivo encontrado en: {dataset_file}")
    
    # Cargamos el dataset crudo
    dataset = load_dataset("text", data_files=dataset_file)
    dataset = dataset["train"].train_test_split(test_size=0.1)
    print("✅ BLOQUE 2 TERMINADO: Dataset cargado en memoria.")
else:
    print(f"❌ ERROR: No encuentro el archivo en {dataset_file}")
    print("Por favor, copia la ruta usando el panel derecho (Copy file path).")

In [ ]:
# --- BLOQUE 3: TOKENIZACIÓN ---
print("⏳ Iniciando tokenización (esto puede tardar unos minutos)...")

model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024, 
        padding="max_length"
    )

# Aquí ocurre la magia lenta
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print(f"📊 Datos de entrenamiento: {len(tokenized_dataset['train'])} ejemplos")
print(f"📊 Datos de prueba: {len(tokenized_dataset['test'])} ejemplos")
print("✅ BLOQUE 3 TERMINADO: Datos listos para entrar al cerebro de la IA.")

In [ ]:
# --- BLOQUE 4: CONFIGURACIÓN DEL MODELO ---
print("⏳ Configurando GPT-2...")
model = GPT2LMHeadModel.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="/kaggle/working/travian_ascii_model",
    overwrite_output_dir=True,
    num_train_epochs=15,            # 15 Épocas (Fuerza Bruta)
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=50,               # Reportar cada 50 pasos
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

print("✅ BLOQUE 4 TERMINADO: Todo listo para despegar.")

In [ ]:
# --- BLOQUE 5: ENTRENAMIENTO Y GUARDADO ---
print("🚀 INICIANDO ENTRENAMIENTO (15 ÉPOCAS)...")
print("Si ves una barra de progreso abajo, está funcionando.")

trainer.train()

print("💾 Guardando modelo final...")
trainer.save_model("/kaggle/working/modelo_final")
tokenizer.save_pretrained("/kaggle/working/modelo_final")

# Crear ZIP para descargar fácil
import shutil
shutil.make_archive("/kaggle/working/modelo_travian_completo", 'zip', "/kaggle/working/modelo_final")

print("✅✅✅ ¡TODO TERMINADO! ✅✅✅")
print("Ve a la sección 'Output' a la derecha, refresca y descarga 'modelo_travian_completo.zip'")

In [ ]:
from google.colab import drive
import shutil
import os

# 1. Conectar Drive
drive.mount('/content/drive')

# 2. Definir rutas
# Ajusta el nombre si tu archivo se llama diferente
ruta_zip_drive = "/content/drive/My Drive/ASCII-GPT/modelo_ascii_pro.pt" 
carpeta_destino = "./modelo_recuperado"

# 3. Descomprimir
print("📦 Descomprimiendo tu modelo previo...")
shutil.unpack_archive(ruta_zip_drive, carpeta_destino, "zip")

# 4. Verificar que el cerebro está ahí
archivos = os.listdir(carpeta_destino)
print(f"✅ Archivos encontrados: {archivos}")

if "config.json" in archivos and ("pytorch_model.bin" in archivos or "model.safetensors" in archivos):
    print("🎉 ¡EL MODELO ESTÁ SANO! Podemos re-entrenar.")
else:
    print("⚠️ Mmm... falta algo. ¿Seguro que se guardó bien?")

In [ ]:
import shutil
import os

# --- CONFIGURACIÓN DE RUTAS EN KAGGLE ---
# 1. Busca la ruta en el panel derecho (copia el path de tu archivo subido)
# Seguramente será algo como: /kaggle/input/mi-modelo-previo/modelo_ascii_pro.pt
# O si subiste el zip: /kaggle/input/mi-modelo-previo/modelo_ascii_pro.zip

# CAMBIA ESTO POR LA RUTA QUE TE SALE A LA DERECHA:
ruta_archivo_kaggle = "/kaggle/input/proto-ascii-art-01/modelo_ascii_pro.pt" 

carpeta_destino = "/kaggle/working/"

print(f"📦 Preparando modelo desde: {ruta_archivo_kaggle}")

# 2. Descomprimir (o copiar si ya es una carpeta)
# Si es un ZIP:
if ruta_archivo_kaggle.endswith(".zip") or ruta_archivo_kaggle.endswith(".pt"): 
    # A veces los .pt de colab son zips disfrazados, probamos descomprimir
    try:
        shutil.unpack_archive(ruta_archivo_kaggle, carpeta_destino, "zip")
        print("✅ Descomprimido correctamente.")
    except:
        # Si falla, quizás no es zip, intentamos copiarlo tal cual
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)
        shutil.copy(ruta_archivo_kaggle, carpeta_destino)
        print("⚠️ No era un ZIP, copiando archivo crudo...")

# 3. Verificar qué tenemos
print(f"📂 Contenido de la carpeta destino:")
print(os.listdir(carpeta_destino))

# VERIFICACIÓN VISUAL
if os.path.exists(f"{carpeta_destino}/config.json"):
    print("🎉 ¡TODO LISTO! El modelo está cargado y listo para re-entrenar.")
else:
    print("⚠️ OJO: No veo el config.json. Revisa si se creó una subcarpeta dentro.")

In [ ]:
import os
from transformers import GPT2LMHeadModel, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# --- 1. DETECTIVE DE CARPETAS 🕵️‍♂️ ---
# Buscamos dónde cayó el config.json
base_path = "/kaggle/working/modelo_recuperado"
subfolder_path = os.path.join(base_path, "modelo_ascii_pro")

if os.path.exists(os.path.join(subfolder_path, "config.json")):
    ruta_real = subfolder_path
    print(f"✅ ¡Encontrado! El modelo está en la subcarpeta: {ruta_real}")
elif os.path.exists(os.path.join(base_path, "config.json")):
    ruta_real = base_path
    print(f"✅ ¡Encontrado! El modelo está en la raíz: {ruta_real}")
else:
    raise FileNotFoundError("❌ Sigo sin encontrar el config.json. Revisa la descompresión.")

# --- 2. CARGAR EL MODELO PREVIO ---
print("🔄 Cargando tu modelo anterior (Transfer Learning)...")
tokenizer = AutoTokenizer.from_pretrained(ruta_real)
model = GPT2LMHeadModel.from_pretrained(ruta_real)

# Asegurar tokens especiales
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Preparar el collator (necesario para el entrenador)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- 3. CONFIGURAR EL ENTRENAMIENTO (FASE 2) ---
training_args = TrainingArguments(
    output_dir="/kaggle/working/travian_ascii_reloaded",
    overwrite_output_dir=True,
    num_train_epochs=5,             # 5 Épocas extra para perfeccionar
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,             # Un poco más bajo para afinar detalles
    weight_decay=0.01,
    warmup_steps=200,
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# --- 4. ¡A ENTRENAR! ---
print("🚀 Iniciando Fase 2: Perfeccionamiento Geométrico...")
trainer.train()

# --- 5. GUARDAR RESULTADO FINAL ---
output_final = "/kaggle/working/modelo_travian_v2"
trainer.save_model(output_final)
tokenizer.save_pretrained(output_final)

# Crear ZIP para descargar
import shutil
shutil.make_archive("/kaggle/working/modelo_travian_v2_completo", 'zip', output_final)

print("✅✅✅ ¡ENTRENAMIENTO COMPLETADO! ✅✅✅")
print("Descarga 'modelo_travian_v2_completo.zip' del panel derecho.")

In [ ]:
import os
from transformers import GPT2LMHeadModel, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

print("🕵️‍♂️ Iniciando búsqueda implacable del 'config.json'...")

# 1. EL SABUESO: Buscamos el archivo recursivamente
root_dir = "/kaggle/working" # Buscamos en todo el espacio de trabajo
ruta_real = None

for dirpath, dirnames, filenames in os.walk(root_dir):
    # print(f"📂 Revisando: {dirpath}") # Descomenta si quieres ver por dónde pasa
    if "config.json" in filenames:
        ruta_real = dirpath
        print(f"\n🎉 ¡ENCONTRADO! El modelo se esconde en: {ruta_real}")
        break

if not ruta_real:
    # SI LLEGAMOS AQUÍ, EL ARCHIVO NO EXISTE (EL ZIP ESTABA MAL O INCOMPLETO)
    print("\n❌ ERROR CRÍTICO: No encontré 'config.json' en ninguna parte.")
    print("Esto significa que el ZIP que subiste a Kaggle NO tenía todos los archivos.")
    print("Archivos encontrados en total:")
    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            print(os.path.join(dirpath, f))
    raise FileNotFoundError("No puedo entrenar sin el config.json")

# --- SI LO ENCONTRÓ, SEGUIMOS AQUÍ ---

print("🔄 Cargando modelo desde la ruta encontrada...")
tokenizer = AutoTokenizer.from_pretrained(ruta_real)
model = GPT2LMHeadModel.from_pretrained(ruta_real)

# Asegurar tokens especiales
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- CONFIGURACIÓN DE ENTRENAMIENTO (FASE 2) ---
training_args = TrainingArguments(
    output_dir="/kaggle/working/travian_ascii_reloaded",
    overwrite_output_dir=True,
    num_train_epochs=5,             # 5 Épocas extra
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=200,
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# --- ¡FUEGO! ---
print("🚀 Arrancando entrenamiento de una puta vez...")
trainer.train()

# --- GUARDAR ---
output_final = "/kaggle/working/modelo_travian_v2"
trainer.save_model(output_final)
tokenizer.save_pretrained(output_final)
import shutil
shutil.make_archive("/kaggle/working/modelo_travian_v2_completo", 'zip', output_final)

print("✅✅✅ ¡HECHO! ✅✅✅")

In [ ]:
import os
from transformers import GPT2LMHeadModel, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# --- 1. LOCALIZADOR DEL MODELO (Basado en tu captura) ---
print("📡 Escaneando ruta exacta del backup...")

# Buscamos la carpeta específica que sale en tu foto
nombre_carpeta_objetivo = "Backup_Modelo_Travian_V1"
ruta_modelo = None

for root, dirs, files in os.walk("/kaggle/input"):
    if nombre_carpeta_objetivo in dirs:
        ruta_modelo = os.path.join(root, nombre_carpeta_objetivo)
        print(f"✅ ¡BLANCO FIJADO! Modelo encontrado en: {ruta_modelo}")
        break

if not ruta_modelo:
    # Si falla la búsqueda exacta, intentamos una ruta manual basada en la foto
    ruta_modelo = "/kaggle/input/proto-ascii-art-001/Backup_Modelo_Travian_V1"
    print(f"⚠️ No lo encontré automáticamente, probando ruta manual: {ruta_modelo}")

# Verificación final de archivos clave
if not os.path.exists(os.path.join(ruta_modelo, "config.json")):
    raise FileNotFoundError(f"❌ Error grave: No veo config.json en {ruta_modelo}")

# --- 2. CARGAR TU MODELO GUARDADO ---
print("🔄 Cargando pesos del modelo (esto puede tardar unos segundos)...")
try:
    tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
    model = GPT2LMHeadModel.from_pretrained(ruta_modelo)
    print("🤖 Modelo cargado exitosamente.")
except Exception as e:
    print(f"❌ Error cargando el modelo: {e}")
    raise e

# Ajustes de tokens
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Verificar que tenemos los datos procesados (del Bloque 3)
if 'tokenized_dataset' not in globals():
    raise NameError("🛑 ¡ALTO! No has corrido el BLOQUE 3 (Tokenización). Sube y córrelo antes de entrenar.")

# --- 3. CONFIGURAR RE-ENTRENAMIENTO (FASE 2) ---
training_args = TrainingArguments(
    output_dir="/kaggle/working/travian_ascii_reloaded",
    overwrite_output_dir=True,
    num_train_epochs=5,              # 5 épocas extra sobre lo que ya sabe
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,              # Rate bajo para refinar, no destruir
    weight_decay=0.01,
    warmup_steps=200,
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# --- 4. ¡A ENTRENAR! ---
print("🚀 Iniciando Re-entrenamiento (Fase 2)...")
trainer.train()

# --- 5. GUARDAR Y EMPAQUETAR ---
output_final = "/kaggle/working/modelo_travian_v2_final"
trainer.save_model(output_final)
tokenizer.save_pretrained(output_final)

import shutil
shutil.make_archive("/kaggle/working/modelo_travian_v2_completo", 'zip', output_final)

print("✅✅✅ ¡MISIÓN CUMPLIDA! ✅✅✅")
print("Ve al panel derecho (Output), refresca y descarga 'modelo_travian_v2_completo.zip'")

In [ ]:
import os
# ESTO CALLA EL AVISO ROJO Y EVITA EL BLOQUEO
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import GPT2LMHeadModel, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# --- 1. LOCALIZADOR DEL MODELO ---
# (Esto ya funcionó, lo dejamos igual)
nombre_carpeta_objetivo = "Backup_Modelo_Travian_V1"
ruta_modelo = None
for root, dirs, files in os.walk("/kaggle/input"):
    if nombre_carpeta_objetivo in dirs:
        ruta_modelo = os.path.join(root, nombre_carpeta_objetivo)
        break
if not ruta_modelo:
    ruta_modelo = "/kaggle/input/proto-ascii-art-001/Backup_Modelo_Travian_V1"

print(f"✅ Usando modelo en: {ruta_modelo}")

# --- 2. CARGAR MODELO ---
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
model = GPT2LMHeadModel.from_pretrained(ruta_modelo)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- 3. CONFIGURAR RE-ENTRENAMIENTO (CORREGIDO) ---
training_args = TrainingArguments(
    output_dir="/kaggle/working/travian_ascii_reloaded",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=200,
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    # 🚨 AQUÍ ESTÁ EL CAMBIO MÁGICO 🚨
    dataloader_num_workers=0  # Ponemos 0 para evitar el deadlock. ¡Esto volará!
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# --- 4. ¡A ENTRENAR RÁPIDO! ---
print("🚀 Iniciando Re-entrenamiento (Modo Turbo)...")
trainer.train()

# --- 5. GUARDAR ---
output_final = "/kaggle/working/modelo_travian_v2_final"
trainer.save_model(output_final)
tokenizer.save_pretrained(output_final)
import shutil
shutil.make_archive("/kaggle/working/modelo_travian_v2_completo", 'zip', output_final)
print("✅✅✅ ¡LISTO! Descarga el ZIP.")

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Mantenemos esto por seguridad

from transformers import GPT2LMHeadModel, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# --- 1. LOCALIZAR MODELO (Igual que antes) ---
nombre_carpeta_objetivo = "Backup_Modelo_Travian_V1"
ruta_modelo = None
for root, dirs, files in os.walk("/kaggle/input"):
    if nombre_carpeta_objetivo in dirs:
        ruta_modelo = os.path.join(root, nombre_carpeta_objetivo)
        break
if not ruta_modelo:
    ruta_modelo = "/kaggle/input/proto-ascii-art-001/Backup_Modelo_Travian_V1"

print(f"✅ Modelo base: {ruta_modelo}")

# --- 2. CARGAR ---
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
model = GPT2LMHeadModel.from_pretrained(ruta_modelo)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- 3. CONFIGURACIÓN REALISTA (2 ÉPOCAS) ---
training_args = TrainingArguments(
    output_dir="/kaggle/working/travian_ascii_reloaded",
    overwrite_output_dir=True,
    num_train_epochs=2,              # <--- CAMBIO CLAVE: Solo 2 vueltas.
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=200,
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    dataloader_num_workers=0
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# --- 4. ENTRENAR ---
# Cálculo rápido: Si va a 0.20 it/s, esto tardará unas 8-9 horas.
# Kaggle aguanta 12h. ¡ESTAMOS DENTRO!
print("🚀 Iniciando Re-entrenamiento (Ajustado a 2 épocas para terminar hoy)...")
trainer.train()

# --- 5. GUARDAR ---
output_final = "/kaggle/working/modelo_travian_v2_final
"
trainer.save_model(output_final)
tokenizer.save_pretrained(output_final)
import shutil
shutil.make_archive("/kaggle/working/modelo_travian_v2_completo", 'zip', output_final)
print("✅✅✅ ¡PROCESO TERMINADO! Descarga el ZIP.")

In [ ]:
# CÓDIGO DE EMERGENCIA: EJECUTAR SOLO SI PARAS EL ENTRENAMIENTO MANUALMENTE
print("🚨 GUARDADO DE EMERGENCIA 🚨")
trainer.save_model("/kaggle/working/modelo_salvado_emergencia")
tokenizer.save_pretrained("/kaggle/working/modelo_salvado_emergencia")

import shutil
shutil.make_archive("/kaggle/working/modelo_emergencia_zip", 'zip', "/kaggle/working/modelo_salvado_emergencia")
print("✅ ¡Salvado! Descarga el ZIP de emergencia.")

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# 1. Cargar el modelo recién horneado
ruta_modelo = "/kaggle/working/modelo_travian_v2_final"

print("🏰 Cargando al Arquitecto Travian...")
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
model = AutoModelForCausalLM.from_pretrained(ruta_modelo)

# Mover a GPU para que sea rápido
model.cuda()

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# 2. Función para imprimir bonito
def imprimir_ascii(texto):
    print("\n" + "⚔️" * 30)
    print(texto.replace("<|startoftext|>", "").replace("<|endoftext|>", ""))
    print("⚔️" * 30 + "\n")

# 3. PROMPTS DE PRUEBA
prompts = [
    "<|startoftext|>\nUser: Dibuja un ASCII art de buildings castle\nAI:\n",
    "<|startoftext|>\nUser: Dibuja un ASCII art de un mapa\nAI:\n",
    "<|startoftext|>\nUser: Dibuja un ASCII art de animals dragons\nAI:\n"
]

print("🎨 GENERANDO ARTE (Momento de la verdad)...\n")

for prompt in prompts:
    print(f"--- Prompt: {prompt.strip()} ---")
    
    output = generator(
        prompt,
        max_new_tokens=800,   # Suficiente para un dibujo grande
        temperature=0.6,      # Bajo para mantener las paredes rectas
        top_p=0.9,
        repetition_penalty=1.2, # IMPORTANTE: Con loss 0.6 puede querer repetir mucho, esto lo frena
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    imprimir_ascii(output[0]["generated_text"])

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Cargamos desde la carpeta descomprimida (que aun esta ahi)
ruta = "/kaggle/working/modelo_travian_v2_final"

print("🏰 Cargando Arquitecto Travian V2...")
tokenizer = AutoTokenizer.from_pretrained(ruta)
model = AutoModelForCausalLM.from_pretrained(ruta).cuda()

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Prompt de prueba
prompt = "<|startoftext|>\nUser: Dibuja un ASCII art de buildings castle\nAI:\n"

print("🎨 Generando Castillo...")
output = generator(
    prompt,
    max_new_tokens=600,
    temperature=0.65,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

print(output[0]["generated_text"])

In [ ]:
# INTENTO DE REANIMACIÓN DE LIBRERÍAS
# Solo haz esto si YA TIENES EL ZIP EN TU PC

!pip install --force-reinstall transformers==4.37.2
!pip install --force-reinstall tokenizers==0.15.2

print("🔄 Librerías reinstaladas. Intentando importar de nuevo...")

# Probamos importar solo lo necesario (sin pipeline para evitar el error de video)
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

try:
    # Ruta de tu modelo
    ruta_modelo = "/kaggle/working/modelo_travian_v2_final"
    
    print("🏰 Cargando modelo a lo bruto...")
    tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
    model = GPT2LMHeadModel.from_pretrained(ruta_modelo).cuda()
    
    print("✅ ¡LOGRADO! El modelo cargó.")
    
    # Generación manual (sin pipeline)
    prompt = "<|startoftext|>\nUser: Dibuja un ASCII art de buildings castle\nAI:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    print("🎨 Pintando...")
    outputs = model.generate(
        **inputs, 
        max_new_tokens=500,
        temperature=0.6,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

except Exception as e:
    print(f"❌ Python sigue borracho: {e}")
    print("CONCLUSIÓN: Vete a dormir con tu ZIP. Mañana lo probamos en limpio.")